In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_DIR_PATH = './data/minet'

In [ ]:
num_classes=7
batch_size=32
IMAGE_SIZE=[224,224]

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg.summary()


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
num_classes=7
prediction = Dense(units=768, activation='relu')(x)
prediction=Dropout(0.4)
prediction=Dense(units=num_classes,activation='softmax')(x)

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,validation_split=0.2)

test_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)

training_set = train_datagen.flow_from_directory('../input/minerals-identification-dataset/minet',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

test_set = test_datagen.flow_from_directory('../input/minerals-identification-dataset/minet',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',subset='validation')

In [ ]:
modelcheckpoint = ModelCheckpoint('/kaggle/working/with_vgg16_1.h5', monitor='val_loss', verbose=1, save_best_only=True, period=1)

In [ ]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=[modelcheckpoint])
